In [2]:
import sys

In [6]:
sys.executable

'C:\\Anaconda3\\envs\\pommenv\\python.exe'

In [7]:
import requests
import pandas as pd
import numpy as np
from io import StringIO

import json
from tqdm import tqdm_notebook as tqdm
from datetime import datetime

In [8]:
from IPython.core.display import display, HTML


In [9]:
url_search = 'https://rating.chgk.info/api/tournaments.json/search?name={}'
url_tournament = 'https://rating.chgk.info/api/tournaments/{}/list'
url_tournament_info = 'https://rating.chgk.info/api/tournaments/{}/'
url_recaps = 'https://rating.chgk.info/api/tournaments/{idtournament}/recaps/{idteam}'
url_player = 'https://rating.chgk.info/api/players/{}'
url_team = 'https://rating.chgk.info/api/teams/{}'

Израиль Германия Россия Беларусь США Великобритания Молдова Латвия Литва Эстония Узбекистан Казахстан Таджикистан

In [10]:
countries = ['Израиль','Германия','Россия','Беларусь','США','Великобритания','Молдова','Латвия','Литва','Эстония','Узбекистан','Казахстан','Таджикистан']

In [11]:
def get_json(url):
    response = requests.get(url).text
    response = json.load(StringIO(response))
    return response

In [12]:
def get_tournament_list(query):
    response = requests.get(url_search.format(query)).text
    response = json.load(StringIO(response))
    return pd.DataFrame(response['items'])

In [56]:
def get_national_championships(countries):
    ts = []
    for country in countries:
        query = 'чемпионат '+ country[:-1]
        t = get_tournament_list(query)
        t = t[t.name.apply(lambda x: x.split()[-1][1:-1] in (country[1:-1], country[1:]))]
        t = pd.DataFrame({'idtournament': t.idtournament, 'year': t.date_end.apply(lambda x: x[:4]), 'country': country})
        ts.append(t)
    return pd.concat(ts).reset_index()

In [60]:
championchips = get_national_championships(["Казахстан"])

In [61]:
championchips

,index,country,idtournament,year
0,0,Казахстан,1874,2011
1,1,Казахстан,2015,2012
2,2,Казахстан,2414,2013
3,3,Казахстан,2848,2014
4,4,Казахстан,3291,2015
5,5,Казахстан,3790,2016
6,6,Казахстан,4235,2017
7,7,Казахстан,5049,2018
8,8,Казахстан,5487,2019


In [19]:
get_json(url_tournament_info.format(716))

[{'archive': None,
  'comment': None,
  'date_archived_at': None,
  'date_end': '1998-04-18 00:00:00',
  'date_requests_allowed_to': None,
  'date_start': '1998-04-18 00:00:00',
  'db_tags': ['http://db.chgk.info/tour/israel98'],
  'discounted_payment_currency': '',
  'discounted_payment_reason': None,
  'discounted_payment_value': None,
  'idtournament': '716',
  'long_name': '',
  'main_payment_currency': '',
  'main_payment_value': None,
  'name': 'Чемпионат Израиля',
  'questions_total': None,
  'site_url': '',
  'tour_count': None,
  'tour_ques_per_tour': None,
  'tour_questions': None,
  'tournament_in_rating': '0',
  'town': 'Тель-Авив',
  'type_name': 'Обычный'}]

In [20]:
def format_tournament_dates(iso_date_start, iso_date_end, fmt="%Y-%m-%d %H:%M:%S"):
    ds = datetime.strptime(iso_date_start, fmt)
    de = datetime.strptime(iso_date_end, fmt)
    # TODO date parser to human-readable dates
    date = de.strftime("%d.%m.%Y")
    if ds.day != de.day:
        date = '{}-{}'.format(ds.strftime("%d"), date)
    return date

In [21]:
format_tournament_dates('2019-06-01 16:00:00', '2019-06-02 22:00:00')

'01-02.06.2019'

In [22]:
get_json(url_tournament.format(5674))

[{'base_name': 'Десятый вал',
  'bonus_a': '0',
  'bonus_b': '773',
  'current_name': 'Десятый вал',
  'diff_bonus': '-35',
  'idteam': '194',
  'included_in_rating': 1,
  'mask': '101110101000111001101010100010110000111010001101100010000111',
  'position': '11.5',
  'predicted_position': '10',
  'questions_total': '29',
  'rating_r': '2813',
  'tech_rating_rb': '2355',
  'tech_rating_rg': '3710',
  'tech_rating_rt': '3710'},
 {'base_name': 'Вист!',
  'bonus_a': '1598',
  'bonus_b': '638',
  'current_name': 'Вист!',
  'diff_bonus': '-43',
  'idteam': '208',
  'included_in_rating': 1,
  'mask': '101100100010111100111011100101110000000010001101001000000000',
  'position': '16.5',
  'predicted_position': '13',
  'questions_total': '24',
  'rating_r': '3591',
  'tech_rating_rb': '4206',
  'tech_rating_rg': '3058',
  'tech_rating_rt': '3582'},
 {'base_name': 'Хохма',
  'bonus_a': '2150',
  'bonus_b': '773',
  'current_name': 'Хохма',
  'diff_bonus': '33',
  'idteam': '261',
  'included_in_r

In [23]:
def get_name(idplayer, format='ns'):
    fdict = dict(n='name', s='surname', p='patronymic')
    response = json.load(StringIO(requests.get(url_player.format(idplayer)).text))[0]
    return ' '.join([response[fdict[f]] for f in format])

def get_names(players):
    return [get_name(i) for i in tqdm(players)]

In [29]:
def get_winner_recaps(idtournament):
    """
    Returns:
    {team: (idteam, current_name, town),
     recaps: [(idplayer, surname name patronymic), ...]}
    """
    # Get tournament results
    teams = get_json(url_tournament.format(idtournament))
    # Get winner info
    winner = [t for t in teams if float(t['position']) < 2]
    if len(winner) > 1:
        print('More than one winner in tournament', idtournament)
    try:
        winner = winner[0]
    except IndexError:
        # Tournament did not happen yet, no winners
        return {'team': None, 'recaps': None}
    idteam, winner = winner['idteam'], winner['current_name']
    town = get_json(url_team.format(idteam))[0]['town']
    # Get winner recaps
    winner_recaps = get_json(url_recaps.format(idtournament=idtournament, idteam=idteam))
    # Format recaps (idplayer, surname name patronymic (K?))
    winner_recaps = [(p['idplayer'], get_name(p['idplayer']) + (' (K)' if p['is_captain'] == '1' else '')) for p in winner_recaps]
    
    return {'team': (idteam, winner, town), 'recaps': winner_recaps}

In [28]:
get_json(url_tournament.format(6114))

[]

In [30]:
get_winner_recaps(6114)

{'recaps': None, 'team': None}

In [33]:
def format_player(player_tuple):
    return '<a href="https://rating.chgk.info/player/{}">{}</a>'.format(*player_tuple)
    
def format_team_recaps(team_recaps):
    if team_recaps['team'] is None:
        return None
    head = '<p>Победитель <a href="https://rating.chgk.info/team/{}"><strong>{} ({})</strong> </a></p>'.format(*team_recaps['team'])
    ul = '\n'.join(['<li>'+format_player(pt) for pt in team_recaps['recaps']])
    ul = '<ul>{}</ul>'.format(ul)
    return '{}\n{}'.format(head, ul)

In [34]:
display(HTML(format_team_recaps(get_winner_recaps(6114))))

<IPython.core.display.HTML object>

In [35]:
def get_tournament_header(idtournament):
    tourn = get_json(url_tournament_info.format(idtournament))[0]
    date = format_tournament_dates(tourn['date_start'], tourn['date_end'])
    return tourn['idtournament'], tourn['long_name'], date, tourn['town']

def format_tournament_header(tournament_tuple):
    return '<p><a href="https://rating.chgk.info/tournament/{}"><strong>{}</strong></a>: {}, {}</p>'.format(*tournament_tuple)

In [38]:
def format_tournament(idtournament):
    head = format_tournament_header(get_tournament_header(idtournament))
    body = format_team_recaps(get_winner_recaps(idtournament))
    return '{}\n{}'.format(head, body if body is not None else '')

def format_tournaments(idtournament_list):
    tl = [format_tournament(idtournament) for idtournament in tqdm(idtournament_list)]
    return '\n'.join(tl)

In [39]:
format_tournament(6114)

'<p><a href="https://rating.chgk.info/tournament/6114"><strong>XIII открытый чемпионат Великобритании по игре "Что? Где? Когда?"</strong></a>: 24.05.2020, Лондон</p>\n'

In [115]:
display(HTML(format_tournaments(championchips.idtournament[:5])))